In [1]:
!pip install -U transformers datasets scikit-learn accelerate


In [2]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import (
    accuracy_score,
    precision_recall_fscore_support,
    roc_auc_score,
)

import torch
from datasets import Dataset
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    TrainingArguments,
    Trainer,
)

/Users/gwenythtan/miniforge3/envs/gwen_env/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
train_path = "/Users/gwenythtan/Desktop/BT4012_FINALPROJ/train.csv"
df = pd.read_csv(train_path)

# Expecting columns: 'url' and 'target'
df = df.dropna(subset=["url", "target"])
df["target"] = df["target"].astype(int)

print("Train shape:", df.shape)
print(df.head())
print("Class balance:\n", df["target"].value_counts(normalize=True))


Train shape: (9143, 2)
                                                 url  target
0  http://cafecitoperez.ivanidzakovic.com/admin/a...       1
1                    https://www.guiadoexcel.com.br/       0
2  https://www.mysummercamps.com/forum/For_Camp_D...       1
3                     https://www.health-island.net/       0
4        https://www.youtube.com/watch?v=0jqwikTk9gw       0
Class balance:
 target
0    0.500055
1    0.499945
Name: proportion, dtype: float64


In [4]:
train_df, val_df = train_test_split(
    df,
    test_size=0.2,
    stratify=df["target"],
    random_state=42,
)

print("Train size:", len(train_df))
print("Val size:", len(val_df))

Train size: 7314
Val size: 1829


In [5]:
train_ds = Dataset.from_pandas(train_df.reset_index(drop=True))
val_ds   = Dataset.from_pandas(val_df.reset_index(drop=True))

keep_cols = ["url", "target"]
train_ds = train_ds.remove_columns(
    [c for c in train_ds.column_names if c not in keep_cols]
)
val_ds = val_ds.remove_columns(
    [c for c in val_ds.column_names if c not in keep_cols]
)

train_ds = train_ds.rename_column("target", "labels")
val_ds   = val_ds.rename_column("target", "labels")

In [6]:
MODEL_NAME = "microsoft/deberta-v3-base"
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

MAX_LEN = 256
def tokenize_function(examples):
    return tokenizer(
        examples["url"],
        padding="max_length",
        truncation=True,
        max_length=MAX_LEN,
    )

train_ds = train_ds.map(tokenize_function, batched=True)
val_ds   = val_ds.map(tokenize_function, batched=True)

train_ds.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])
val_ds.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])


/Users/gwenythtan/miniforge3/envs/gwen_env/lib/python3.11/site-packages/transformers/convert_slow_tokenizer.py:564: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(
Map: 100%|██████████| 1829/1829 [00:00<00:00, 7297.32 examples/s]


In [7]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

model = AutoModelForSequenceClassification.from_pretrained(
    MODEL_NAME,
    num_labels=2,
).to(device)

Using device: cpu


Some weights of DebertaV2ForSequenceClassification were not initialized from the model checkpoint at microsoft/deberta-v3-base and are newly initialized: ['classifier.bias', 'classifier.weight', 'pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [8]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = np.argmax(logits, axis=-1)

    acc = accuracy_score(labels, preds)
    precision, recall, f1, _ = precision_recall_fscore_support(
        labels, preds, average="binary", zero_division=0
    )

    try:
        auc = roc_auc_score(labels, logits[:, 1])
    except ValueError:
        auc = float("nan")

    return {
        "accuracy": acc,
        "precision": precision,
        "recall": recall,
        "f1": f1,
        "roc_auc": auc,
    }


In [9]:
output_dir = "./deberta_checkpoints"

from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir=output_dir,
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=32,
    num_train_epochs=2,
    weight_decay=0.01,
    logging_steps=50,
    report_to="none",
)


In [10]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_ds,
    eval_dataset=val_ds,
    compute_metrics=compute_metrics,
)


In [11]:
trainer.train()

/Users/gwenythtan/miniforge3/envs/gwen_env/lib/python3.11/site-packages/torch/utils/data/dataloader.py:668: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


KeyboardInterrupt: 

In [ ]:
metrics = trainer.evaluate()
print("Validation metrics:")
for k, v in metrics.items():
    print(f"{k}: {v}")